In [1]:
# How do you identify and remove duplicate rows based on a subset of columns, keeping the row with the latest timestamp?

In [11]:
import pandas as pd
import numpy as np

data = {
    'user_id':    [101, 101, 102, 102, 103, 103, 104, 105, 105, 106],
    'product_id': [501, 501, 502, 502, 503, 503, 504, 505, 505, 507],
    'rating':     [4,   4,   3,   3,   5,   5,   2,   1,   1,   5],
    'timestamp': [
        '2023-10-01 10:00:00',  # duplicate of row 2 in subset
        '2023-10-01 12:00:00',  # keep this one
        '2023-09-29 09:30:00',  # duplicate of row 4 in subset
        '2023-09-30 09:45:00',  # keep this one
        '2023-09-30 08:00:00',  # duplicate of row 6 in subset
        '2023-10-01 08:15:00',  # keep this one
        '2023-09-28 15:45:00',
        '2023-09-27 11:00:00',  # duplicate of row 9 in subset
        '2023-09-28 11:00:00',  # keep this one
        '2023-10-02 10:00:00'
    ]
}

df = pd.DataFrame(data)
df['timestamp'] = pd.to_datetime(df['timestamp'])

print(df)


   user_id  product_id  rating           timestamp
0      101         501       4 2023-10-01 10:00:00
1      101         501       4 2023-10-01 12:00:00
2      102         502       3 2023-09-29 09:30:00
3      102         502       3 2023-09-30 09:45:00
4      103         503       5 2023-09-30 08:00:00
5      103         503       5 2023-10-01 08:15:00
6      104         504       2 2023-09-28 15:45:00
7      105         505       1 2023-09-27 11:00:00
8      105         505       1 2023-09-28 11:00:00
9      106         507       5 2023-10-02 10:00:00


In [9]:
subset_df = df.groupby(['user_id',
                        'product_id',
                        'rating']).agg({ 'timestamp' : 'max'}).reset_index()

In [8]:
subset_df

,user_id,product_id,rating,timestamp
0,101,501,4,2023-10-01 10:00:00
1,102,502,3,2023-09-29 09:30:00
2,103,503,5,2023-09-30 08:00:00
3,104,504,2,2023-09-28 15:45:00
4,105,505,1,2023-09-27 11:00:00
5,106,507,5,2023-10-02 10:00:00


In [12]:
# Given a column with mixed data types (strings, numbers, NaNs), clean it to extract only numeric values
import pandas as pd
import numpy as np

data = {
    'mixed_column': [
        '123',        # string of a number
        456,          # actual integer
        '78.9',       # string of a float
        'abc',        # non-numeric string
        np.nan,       # NaN
        '1000xyz',    # string with numbers and letters
        'NaN',        # string that looks like NaN
        34.5,         # float
        '  789  ',    # number with spaces
        None          # NoneType
    ]
}

df = pd.DataFrame(data)
df


,mixed_column
0,123
1,456
2,78.9
3,abc
4,NaN
5,1000xyz
6,NaN
7,34.5
8,789
9,None


In [17]:
# Method to extract only integers
df['mixed_column_int'] = pd.to_numeric(df['mixed_column'], errors='coerce')

In [18]:
df

,mixed_column,mixed_column_int
0,123,123.0
1,456,456.0
2,78.9,78.9
3,abc,NaN
4,NaN,NaN
5,1000xyz,NaN
6,NaN,NaN
7,34.5,34.5
8,789,789.0
9,None,NaN


In [31]:
# Method to extract the integer part if exits
import re
def extract_integer(value):
    if pd.isna(value):
        return np.nan
    else:
        match = re.search(r'\d+\.?\d*', value)
        return (match.group()) if match else np.nan

In [32]:
df['extract_int'] = df['mixed_column'].astype(str).apply(extract_integer)

In [33]:
df

,mixed_column,mixed_column_int,extract_int
0,123,123.0,123
1,456,456.0,456
2,78.9,78.9,78.9
3,abc,NaN,NaN
4,NaN,NaN,NaN
5,1000xyz,NaN,1000
6,NaN,NaN,NaN
7,34.5,34.5,34.5
8,789,789.0,789
9,None,NaN,NaN


In [36]:
# How do you convert multiple categorical columns into one-hot encoded features efficiently?
data = {
    'Color': ['Red', 'Blue', 'Green', 'Red', 'Green'],
    'Size': ['S', 'M', 'L', 'M', 'S'],
    'Brand': ['Nike', 'Adidas', 'Puma', 'Nike', 'Adidas'],
    'Price': [100, 150, 200, 130, 170],
    'Rating': [4.5, 4.0, 3.5, 5.0, 4.2]
}

df = pd.DataFrame(data)

In [38]:
df

,Color,Size,Brand,Price,Rating
0,Red,S,Nike,100,4.5
1,Blue,M,Adidas,150,4.0
2,Green,L,Puma,200,3.5
3,Red,M,Nike,130,5.0
4,Green,S,Adidas,170,4.2


In [41]:
pd.get_dummies(df, columns=['Color', 'Size', 'Brand'], drop_first=True, dtype='int')

,Price,Rating,Color_Green,Color_Red,Size_M,Size_S,Brand_Nike,Brand_Puma
0,100,4.5,0,1,0,1,1,0
1,150,4.0,0,0,1,0,0,0
2,200,3.5,1,0,0,0,0,1
3,130,5.0,0,1,1,0,1,0
4,170,4.2,1,0,0,1,0,0


In [42]:
# You have a wide-format DataFrame. Write code to unpivot it to a long format using melt() and then pivot it back.
data = {
    'Student': ['Alice', 'Bob', 'Charlie'],
    'Math': [85, 90, 95],
    'Science': [88, 92, 94],
    'English': [78, 85, 88]
}

df = pd.DataFrame(data)
df


,Student,Math,Science,English
0,Alice,85,88,78
1,Bob,90,92,85
2,Charlie,95,94,88


In [46]:
long_format = pd.melt(df,id_vars='Student', var_name='Subject', value_name='Marks')
long_format

,Student,Subject,Marks
0,Alice,Math,85
1,Bob,Math,90
2,Charlie,Math,95
3,Alice,Science,88
4,Bob,Science,92
5,Charlie,Science,94
6,Alice,English,78
7,Bob,English,85
8,Charlie,English,88


In [50]:
long_format.pivot(columns='Subject', values='Marks', index='Student').reset_index()

Subject,Student,English,Math,Science
0,Alice,78,85,88
1,Bob,85,90,92
2,Charlie,88,95,94


In [59]:
# How do you fill missing values in a column using the median value grouped by another column?

data = {
    'Department': ['Sales', 'Sales', 'HR', 'HR', 'IT', 'IT', 'Sales', 'HR', 'IT'],
    'Salary': [50000, np.nan, 40000, 42000, np.nan, 52000, 48000, np.nan, 51000]
}

df = pd.DataFrame(data)
df


,Department,Salary
0,Sales,50000.0
1,Sales,NaN
2,HR,40000.0
3,HR,42000.0
4,IT,NaN
5,IT,52000.0
6,Sales,48000.0
7,HR,NaN
8,IT,51000.0


In [60]:
df.groupby('Department')['Salary'].median()

Department
HR       41000.0
IT       51500.0
Sales    49000.0
Name: Salary, dtype: float64

In [61]:
df.groupby('Department')['Salary'].transform('median')

0    49000.0
1    49000.0
2    41000.0
3    41000.0
4    51500.0
5    51500.0
6    49000.0
7    41000.0
8    51500.0
Name: Salary, dtype: float64

In [62]:
df['Salary'] =df['Salary'].fillna(df.groupby('Department')['Salary'].transform('median'))

In [63]:
df

,Department,Salary
0,Sales,50000.0
1,Sales,49000.0
2,HR,40000.0
3,HR,42000.0
4,IT,51500.0
5,IT,52000.0
6,Sales,48000.0
7,HR,41000.0
8,IT,51000.0


In [65]:
# Extract week number, day name, and whether a date is a weekend from a datetime column.

data = {
    'purchase_id': [101, 102, 103, 104, 105],
    'purchase_date': [
        '2024-12-30',  # Monday
        '2025-01-01',  # Wednesday
        '2025-01-04',  # Saturday
        '2025-01-05',  # Sunday
        '2025-01-06'   # Monday
    ]
}

df = pd.DataFrame(data)
df['purchase_date'] = pd.to_datetime(df['purchase_date'])
df


,purchase_id,purchase_date
0,101,2024-12-30
1,102,2025-01-01
2,103,2025-01-04
3,104,2025-01-05
4,105,2025-01-06


In [68]:
df['week_number'] = df['purchase_date'].dt.isocalendar().week
df

,purchase_id,purchase_date,week_number
0,101,2024-12-30,1
1,102,2025-01-01,1
2,103,2025-01-04,1
3,104,2025-01-05,1
4,105,2025-01-06,2


In [69]:
df['name_of_day'] = df['purchase_date'].dt.day_name()
df

,purchase_id,purchase_date,week_number,name_of_day
0,101,2024-12-30,1,Monday
1,102,2025-01-01,1,Wednesday
2,103,2025-01-04,1,Saturday
3,104,2025-01-05,1,Sunday
4,105,2025-01-06,2,Monday


In [70]:
df['is_weekend'] = df['purchase_date'].dt.weekday >4

In [71]:
df

,purchase_id,purchase_date,week_number,name_of_day,is_weekend
0,101,2024-12-30,1,Monday,False
1,102,2025-01-01,1,Wednesday,False
2,103,2025-01-04,1,Saturday,True
3,104,2025-01-05,1,Sunday,True
4,105,2025-01-06,2,Monday,False


In [80]:
# You have a dataset with an invoice date and delivery date. Calculate business days between them.
data = {
    'order_id': [101, 102, 103, 104, 105],
    'invoice_date': [
        '2024-12-30',  # Monday
        '2025-01-01',  # Wednesday (New Year)
        '2025-01-03',  # Friday
        '2025-01-04',  # Saturday
        '2025-01-06'   # Monday
    ],
    'delivery_date': [
        '2025-01-03',  # Friday
        '2025-01-07',  # Tuesday
        '2025-01-08',  # Wednesday
        '2025-01-08',  # Wednesday
        '2025-01-10'   # Friday
    ]
}

df = pd.DataFrame(data)
df['invoice_date'] = pd.to_datetime(df['invoice_date'])
df['delivery_date'] = pd.to_datetime(df['delivery_date'])
# ✅ Convert datetime columns to numpy datetime64[D] (day-level precision)
start_dates = df['invoice_date'].values.astype('datetime64[D]')
end_dates = df['delivery_date'].values.astype('datetime64[D]')


In [81]:
df['business_days'] = np.busday_count(start_dates, end_dates)

In [82]:
df

,order_id,invoice_date,delivery_date,business_days
0,101,2024-12-30,2025-01-03,4
1,102,2025-01-01,2025-01-07,4
2,103,2025-01-03,2025-01-08,3
3,104,2025-01-04,2025-01-08,2
4,105,2025-01-06,2025-01-10,4


In [83]:
# Given a daily time series with missing dates, fill in all missing dates and forward-fill the values.
data = {
    'date': [
        '2025-01-01',
        '2025-01-02',
        '2025-01-04',  # 2025-01-03 is missing
        '2025-01-06',  # 2025-01-05 is missing
        '2025-01-07'
    ],
    'value': [100, 110, 120, 130, 140]
}

df = pd.DataFrame(data)
df['date'] = pd.to_datetime(df['date'])
df

,date,value
0,2025-01-01,100
1,2025-01-02,110
2,2025-01-04,120
3,2025-01-06,130
4,2025-01-07,140


In [86]:
df.set_index('date', inplace=True)

In [89]:
range= pd.date_range(df.index.min(), df.index.max())
range

DatetimeIndex(['2025-01-01', '2025-01-02', '2025-01-03', '2025-01-04',
               '2025-01-05', '2025-01-06', '2025-01-07'],
              dtype='datetime64[ns]', freq='D')

In [91]:
df = df.reindex(range).ffill()

In [92]:
df

,value
2025-01-01,100.0
2025-01-02,110.0
2025-01-03,110.0
2025-01-04,120.0
2025-01-05,120.0
2025-01-06,130.0
2025-01-07,140.0


In [ ]:
# Calculate the rolling 7-day average of sales per store and align it to the current date.

In [94]:
# Create a date range for 20 days
dates = pd.date_range(start='2025-01-01', end='2025-01-20')

# Create sample data for 2 stores
data = {
    'date': list(dates) * 2,
    'store_id': ['Store_A'] * len(dates) + ['Store_B'] * len(dates),
    'sales': np.random.randint(100, 500, size=len(dates) * 2)
}

df = pd.DataFrame(data)

# Shuffle rows to remove order (optional)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
df


,date,store_id,sales
0,2025-01-20,Store_A,204
1,2025-01-17,Store_A,101
2,2025-01-16,Store_A,396
3,2025-01-07,Store_B,275
4,2025-01-05,Store_A,327
5,2025-01-13,Store_A,257
6,2025-01-18,Store_B,470
7,2025-01-08,Store_B,243
8,2025-01-20,Store_B,235
9,2025-01-07,Store_A,242


In [101]:
df = df.sort_values(by=['store_id', 'date'])
df.groupby('store_id')['sales'].rolling(window=7, min_periods=1).mean()

store_id    
Store_A   18    459.000000
          22    300.500000
          25    235.000000
          28    287.750000
          4     295.600000
          20    297.666667
          9     289.714286
          36    241.857143
          15    281.857143
          11    334.285714
          32    308.428571
          21    301.285714
          5     294.000000
          12    324.285714
          37    357.857143
          2     354.142857
          1     301.285714
          16    334.428571
          34    334.428571
          0     326.857143
Store_B   35    249.000000
          24    298.000000
          33    312.000000
          30    276.500000
          17    261.000000
          10    299.333333
          3     295.857143
          7     295.000000
          38    304.571429
          23    308.714286
          26    338.285714
          13    330.428571
          31    301.142857
          19    285.857143
          14    292.857143
          29    251.000000
          27   